In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-predict-chemical-bindings/sample_submission.csv
/kaggle/input/leash-predict-chemical-bindings/train.parquet
/kaggle/input/leash-predict-chemical-bindings/test.parquet
/kaggle/input/leash-predict-chemical-bindings/train.csv
/kaggle/input/leash-predict-chemical-bindings/test.csv
/kaggle/input/train-data-featurized/brd4_train_data.pt
/kaggle/input/train-data-featurized/seh_train_data.pt
/kaggle/input/train-data-featurized/hsa_train_data.pt


## Introduction

In this tutorial, we will go through another of the several representations of molecules for ML and AI: Molecular Graphs. Using PyTorch Geometric, we are able to convert molecular data from their standard SMILES strings representation into actual graphs with nodes and edges representing atoms and bonds.

We first start by loading all the necessary packages:
1. DuckDB is used to load in the datasets from the parquet files.
2. RDKit is a cheminformatics library which will help in the conversion process from SMILES to graphs.
3. We install Torch for all our neural network needs.

In [3]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 1.9 MB/s eta 0:00:0000:0100:01


In [4]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 20.0 MB/s eta 0:00:0000:0100:01


In [5]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 628.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.6 MB/s eta 0:00:0000:0100:01


In [6]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'

con = duckdb.connect()

# Define the number of samples you want per category (binders and non-binders) per protein
samples_per_category = 1000  # Adjust this number as needed, this is just an example for the tutorial. 
#Your real values should be higher than this for proper training.

def get_balanced_data_for_protein(file_path, protein, samples):
    """
    Fetches a balanced dataset for a specific protein.
    
    Parameters:
    - file_path: Path to the dataset file.
    - protein: The name of the protein.
    - samples: Number of samples per binder category (1 or 0).
    
    Returns:
    - A pandas DataFrame containing the balanced dataset for the protein.
    """
    query = f"""
    (SELECT * FROM parquet_scan('{file_path}')
     WHERE binds = 0 AND protein_name = '{protein}'
     ORDER BY random()
     LIMIT {samples})
    UNION ALL
    (SELECT * FROM parquet_scan('{file_path}')
     WHERE binds = 1 AND protein_name = '{protein}'
     ORDER BY random()
     LIMIT {samples})
    """
    return con.query(query).df()

# List of proteins to query for
proteins = ['sEH', 'BRD4', 'HSA']

# Creating a dictionary of dataframes, one for each protein
datasets = {}
for protein in proteins:
    datasets[protein] = get_balanced_data_for_protein(train_path, protein, samples_per_category)

# At this point, `datasets` contains separate dataframes for sEH, BRD4, and HSA
# For example, to access the dataset for sEH:
seh_df = datasets['sEH']

# And similarly for BRD4 and HSA
brd4_df = datasets['BRD4']
hsa_df = datasets['HSA']


'''We now have balanced data for training for each protein, 
which we will featurize for our GNN model'''

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

'We now have balanced data for training for each protein, \nwhich we will featurize for our GNN model'

In [7]:
'''We do a similar process to load in the test datasets, 
however this time we only filter for the proteins, and
we don't balance them'''
def get_data_for_protein(file_path, protein):
    """
    Fetches the dataset for a specific protein from the test dataset.
    
    Parameters:
    - file_path: Path to the dataset file.
    - protein: The name of the protein.
    
    Returns:
    - A pandas DataFrame containing the dataset for the protein.
    """
    query = f"""
    SELECT * FROM parquet_scan('{file_path}')
    WHERE protein_name = '{protein}'
    """
    return con.query(query).df()

test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

# Assuming the connection `con` to DuckDB is still open from the previous operations

# Creating a dictionary to store the filtered test datasets
test_datasets = {}
for protein in proteins:  # Using the same list of proteins: ['sEH', 'BRD4', 'HSA']
    test_datasets[protein] = get_data_for_protein(test_path, protein)

# At this point, `test_datasets` contains separate dataframes for sEH, BRD4, and HSA without balancing
# For example, to access the test dataset for sEH:
seh_test_df = test_datasets['sEH']

# And similarly for BRD4 and HSA
brd4_test_df = test_datasets['BRD4']
hsa_test_df = test_datasets['HSA']
con.close()

In [25]:
seh_df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,288273434,O=C(O)[C@H]1CC2CCCCC2N1C(=O)OCC1c2ccccc2-c2ccc...,Cl.NCc1ccnc(C(N)=O)c1,COc1nc(C)ccc1CN,COc1nc(C)ccc1CNc1nc(NCc2ccnc(C(N)=O)c2)nc(N2C3...,sEH,0
1,43070723,CSc1ncc(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)O)n1,Cl.NCC=C(Cl)Cl,Cc1ccc(Cl)c(N)c1,CSc1ncc(Nc2nc(NCC=C(Cl)Cl)nc(Nc3cc(C)ccc3Cl)n2...,sEH,0
2,72963530,O=C(NC(CC1CCCCC1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1ncc(Cl)nc1Cl,Nc1nc2ccc(Cl)cc2s1,O=C(N[Dy])C(CC1CCCCC1)Nc1nc(Nc2nc3ccc(Cl)cc3s2...,sEH,0
3,42485921,CS(=O)(=O)c1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3...,Nc1noc2ccc(F)cc12,Nc1cccc(N2CCOCC2)c1,CS(=O)(=O)c1ccc(C(=O)N[Dy])c(Nc2nc(Nc3cccc(N4C...,sEH,0
4,251849009,O=C(O)C[C@@H](Cc1cccc(F)c1)NC(=O)OCC1c2ccccc2-...,COC(=O)c1cc(OC)c(OC)cc1N,CC1(C)CC(CN)C(C)(C)O1,COC(=O)c1cc(OC)c(OC)cc1Nc1nc(NCC2CC(C)(C)OC2(C...,sEH,0


In [26]:
seh_test_df.head() #note we don't have a binds column here

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
0,295246832,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,sEH
1,295246835,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,sEH
2,295246838,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC1(C)CCCC1(O)CN,C#CCCC[C@H](Nc1nc(NCC2(O)CCCC2(C)C)nc(Nc2ccc(C...,sEH
3,295246841,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,COC(=O)c1cc(Cl)sc1N,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2sc(Cl)c...,sEH
4,295246844,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CSC1CCC(CN)CC1,C#CCCC[C@H](Nc1nc(NCC2CCC(SC)CC2)nc(Nc2ccc(C=C...,sEH


## Featurizing

In [8]:
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [9]:
# Atom Featurisation
## Auxiliary function for one-hot enconding transformation based on list of
##permitted values

def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """
    if x not in permitted_list:
        x = permitted_list[-1]
    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]
    return binary_encoding
    
    
# Main atom feat. func

def get_atom_features(atom, use_chirality=True):
    # Define a simplified list of atom types
    permitted_atom_types = ['C', 'N', 'O', 'S', 'P', 'F', 'Cl', 'Br', 'I','Dy', 'Unknown']
    atom_type = atom.GetSymbol() if atom.GetSymbol() in permitted_atom_types else 'Unknown'
    atom_type_enc = one_hot_encoding(atom_type, permitted_atom_types)
    
    # Consider only the most impactful features: atom degree and whether the atom is in a ring
    atom_degree = one_hot_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 'MoreThanFour'])
    is_in_ring = [int(atom.IsInRing())]
    
    # Optionally include chirality
    if use_chirality:
        chirality_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_features = atom_type_enc + atom_degree + is_in_ring + chirality_enc
    else:
        atom_features = atom_type_enc + atom_degree + is_in_ring
    
    return np.array(atom_features, dtype=np.float32)

In [10]:
# Bond featurization

def get_bond_features(bond):
    # Simplified list of bond types
    permitted_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC, 'Unknown']
    bond_type = bond.GetBondType() if bond.GetBondType() in permitted_bond_types else 'Unknown'
    
    # Features: Bond type, Is in a ring
    features = one_hot_encoding(bond_type, permitted_bond_types) \
               + [int(bond.IsInRing())]
    
    return np.array(features, dtype=np.float32)

In [11]:
def create_pytorch_geometric_graph_data_list_from_smiles_and_labels(x_smiles, ids, y=None):
    data_list = []
    
    for index, smiles in enumerate(x_smiles):
        mol = Chem.MolFromSmiles(smiles)
        
        if not mol:  # Skip invalid SMILES strings
            continue
        
        # Node features
        atom_features = [get_atom_features(atom) for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features, dtype=torch.float)
        
        # Edge features
        edge_index = []
        edge_features = []
        for bond in mol.GetBonds():
            start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            edge_index += [(start, end), (end, start)]  # Undirected graph
            bond_feature = get_bond_features(bond)
            edge_features += [bond_feature, bond_feature]  # Same features in both directions
        
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_features, dtype=torch.float)
        
        # Creating the Data object
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
        data.molecule_id = ids[index]
        if y is not None:
            data.y = torch.tensor([y[index]], dtype=torch.float)
        
        data_list.append(data)
    
    return data_list

In [12]:
from tqdm import tqdm
def featurize_data_in_batches(smiles_list, labels_list, batch_size):
    data_list = []
    # Define tqdm progress bar
    pbar = tqdm(total=len(smiles_list), desc="Featurizing data")
    for i in range(0, len(smiles_list), batch_size):
        smiles_batch = smiles_list[i:i+batch_size]
        labels_batch = labels_list[i:i+batch_size]
        ids_batch = ids_list[i:i+batch_size]
        batch_data_list = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(smiles_batch, ids_batch, labels_batch)
        data_list.extend(batch_data_list)
        pbar.update(len(smiles_batch))
        
    pbar.close()
    return data_list

In [ ]:
# Define the batch size for featurization
batch_size = 2**8
# List of proteins and their corresponding dataframes
proteins_data = {
    'sEH': seh_df,
    'BRD4': brd4_df,
    'HSA': hsa_df
}
# Dictionary to store the featurized data for each protein
featurized_data = {}
# Loop over each protein and its dataframe
for protein_name, df in proteins_data.items():
    print(f"Processing {protein_name}...")
    smiles_list = df['molecule_smiles'].tolist()
    ids_list = df['id'].tolist()
    labels_list = df['binds'].tolist()
 # Featurize the data
    featurized_data[protein_name] = featurize_data_in_batches(smiles_list, labels_list, batch_size)
    

seh_train_data = featurized_data['sEH']
brd4_train_data = featurized_data['BRD4']
hsa_train_data = featurized_data['HSA']

At this stage, you can use torch.save to save your featurized datasets

What follows is the featurizing process for the test data. This can also be done in a loop rather than breaking it up into many cells as done here.

In [ ]:
batch_size = 2**8
smiles_list  = brd4_test_df['molecule_smiles'].tolist()
ids_list = brd4_test_df['id'].tolist()
labels_list = [-1]*len(smiles_list) #we dont have the actual labels, so we assign some dummy label list for the function. (don't choose 0 or 1)
brd4_test_data = featurize_data_in_batches(smiles_list, labels_list, batch_size)

In [ ]:
batch_size = 2**8
smiles_list  = seh_test_df['molecule_smiles'].tolist()
ids_list = seh_test_df['id'].tolist()
labels_list = [-1]*len(smiles_list)
seh_test_data = featurize_data_in_batches(smiles_list, labels_list, batch_size)

In [ ]:
batch_size = 2**8
smiles_list  = hsa_test_df['molecule_smiles'].tolist()
ids_list = hsa_test_df['id'].tolist()
labels_list = [-1]*len(smiles_list)
hsa_test_data = featurize_data_in_batches(smiles_list, labels_list, batch_size)

In [24]:
seh_test_data[0] #example of what a pytorch object looks like

Data(x=[35, 22], edge_index=[2, 74], edge_attr=[74, 6], molecule_id=295246832, y=[1])

## Training and Test

In [17]:
featurized_data_train = {
    'sEH': seh_train_data,
    'BRD4': brd4_train_data,
    'HSA': hsa_train_data
}

featurized_data_test = {
    'sEH': seh_test_data,
    'BRD4': brd4_test_data,
    'HSA': hsa_test_data
}

In [18]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing, global_mean_pool, global_max_pool
from torch.nn import BCEWithLogitsLoss
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
import numpy as np

#Define custom GNN layer
class CustomGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(CustomGNNLayer, self).__init__(aggr='max')
        self.lin = nn.Linear(in_channels + 6, out_channels)

    def forward(self, x, edge_index, edge_attr):
        # Start propagating messages
        return MessagePassing.propagate(self, edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_j, edge_attr):
        combined = torch.cat((x_j, edge_attr), dim=1)
        return combined

    def update(self, aggr_out):
        return self.lin(aggr_out)

#Define GNN Model
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout_rate):
        super(GNNModel, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList([CustomGNNLayer(input_dim if i == 0 else hidden_dim, hidden_dim) for i in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)
        self.bns = nn.ModuleList([nn.BatchNorm1d(hidden_dim) for _ in range(num_layers)])
        self.lin = nn.Linear(hidden_dim, 1)
        
    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_attr)
            x = self.bns[i](x)
            x = F.relu(x)
            x = self.dropout(x)


        x = global_max_pool(x, data.batch) # Global pooling to get a graph-level representation
        x = self.lin(x)
        return x


In [19]:
def train_model(train_loader, num_epochs, input_dim, hidden_dim, num_layers, dropout_rate, lr):
    model = GNNModel(input_dim, hidden_dim, num_layers, dropout_rate)
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    criterion = BCEWithLogitsLoss()
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            out = model(batch)
            loss = criterion(out, batch.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}')
    
    return model

def predict_with_model(model, test_loader):
    model.eval()
    predictions = []
    molecule_ids = []

    with torch.no_grad():
        for data in test_loader:
            output = torch.sigmoid(model(data))
            predictions.extend(output.view(-1).tolist())
            molecule_ids.extend(data.molecule_id)

    return molecule_ids, predictions


In [20]:
from torch_geometric.loader import DataLoader

# Other setup remains the same as before
proteins = ['sEH', 'BRD4', 'HSA']
all_predictions = []

for protein in proteins:
    print(f"Training and predicting for {protein}")
    
    # Create DataLoaders for the current protein
    train_loader = DataLoader(featurized_data_train[protein], batch_size=32, shuffle=True)
    test_loader = DataLoader(featurized_data_test[protein], batch_size=32, shuffle=False)
    
    # Train model
    input_dim = train_loader.dataset[0].num_node_features
    hidden_dim = 64
    num_epochs = 11
    num_layers = 4 #Should ideally be set so that all nodes can communicate with each other
    dropout_rate = 0.3
    lr = 0.001
    #These are just example values, feel free to play around with them.
    model = train_model(train_loader,num_epochs, input_dim, hidden_dim,num_layers, dropout_rate, lr)
    
    # Predict
    molecule_ids, predictions = predict_with_model(model, test_loader)
    
    # Collect predictions
    protein_predictions = pd.DataFrame({
        'id': molecule_ids,
        'binds': predictions,
    })
    all_predictions.append(protein_predictions)

Training and predicting for sEH
Epoch 1/11, Loss: 0.6499347899641309
Epoch 2/11, Loss: 0.5018269027036334
Epoch 3/11, Loss: 0.47002418315599837
Epoch 4/11, Loss: 0.4490280719030471
Epoch 5/11, Loss: 0.4219267576459854
Epoch 6/11, Loss: 0.4082246349444465
Epoch 7/11, Loss: 0.3888401493193611
Epoch 8/11, Loss: 0.3691307765150827
Epoch 9/11, Loss: 0.3662852304322379
Epoch 10/11, Loss: 0.3381329883658697
Epoch 11/11, Loss: 0.3507239090071784
Training and predicting for BRD4
Epoch 1/11, Loss: 0.7223644256591797
Epoch 2/11, Loss: 0.6333745131416927
Epoch 3/11, Loss: 0.5858957441080184
Epoch 4/11, Loss: 0.5644472023797413
Epoch 5/11, Loss: 0.5355671795587691
Epoch 6/11, Loss: 0.5255049992175329
Epoch 7/11, Loss: 0.5056566203397418
Epoch 8/11, Loss: 0.48802152796397136
Epoch 9/11, Loss: 0.4698056204924508
Epoch 10/11, Loss: 0.47309916454648215
Epoch 11/11, Loss: 0.4494760783891829
Training and predicting for HSA
Epoch 1/11, Loss: 0.6616935682675195
Epoch 2/11, Loss: 0.6107415642057147
Epoch 3/

In [21]:
# Combine all predictions into one DataFrame
final_predictions = pd.concat(all_predictions, ignore_index=True)
# Convert 'molecule_id' from tensors to integers directly within the DataFrame
final_predictions['id'] = final_predictions['id'].apply(lambda x: x.item())

# Now save the modified DataFrame to a CSV file
final_predictions.to_csv('final_predictions.csv', index=False)

The final predictions csv will be our final output for the model.

In [22]:
final_predictions.head()

,id,binds
0,295246832,0.423596
1,295246835,0.475382
2,295246838,0.373294
3,295246841,0.394595
4,295246844,0.803482
